In [1]:
import os

import pytorch_lightning as pl
import torch
from dotenv import load_dotenv

load_dotenv("../.env")

dataset_folder = os.environ["DATASET_FOLDER"]

In [3]:
from pl_bolts.datamodules.vision_datamodule import VisionDataModule

In [7]:
from typing import Any, Callable, Optional, Union

In [14]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder


class Logo2kDataModule(VisionDataModule):
    name = "Logo2k+"
    dataset_cls = ImageFolder
    dims = (3, 256, 256)

    def __init__(
        self,
        data_dir: Optional[str] = None,
        val_split: Union[int, float] = 0.2,
        num_workers: int = 16,
        normalize: bool = False,
        batch_size: int = 32,
        seed: int = 42,
        shuffle: bool = False,
        pin_memory: bool = False,
        drop_last: bool = False,
        *args: Any,
        **kwargs: Any,
    ) -> None:
        super().__init__(  # type: ignore[misc]
            data_dir=data_dir,
            val_split=val_split,
            num_workers=num_workers,
            normalize=normalize,
            batch_size=batch_size,
            seed=seed,
            shuffle=shuffle,
            pin_memory=pin_memory,
            drop_last=drop_last,
            *args,
            **kwargs,
        )

    @property
    def num_classes(self) -> int:
        return 10

    def default_transforms(self) -> Callable:
        if self.normalize:
            # Normalization mean and std taken from reference implementation
            # https://github.com/msn199959/Logo-2k-plus-Dataset/tree/master/Drna-master
            return transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
                    ),
                ]
            )
        else: 
            return transforms.Compose([transforms.ToTensor()])

In [16]:
dm = Logo2kDataModule(dataset_folder)

In [19]:
dm.setup()

TypeError: __init__() got an unexpected keyword argument 'train'

In [ ]:
class Logo2kTransferLearning(pl.LightningModule):
    def __init__(self):
        